# Assignment 3.3

Group: Mathias Husted, Luciana Amarante, Leonie Kochs

We'll start by importing our functions from last time.

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def lagrange(x, nodes, k):
    f = 1 # Neutral element for multiplication
    for i in range(0, len(nodes)):
        if i != k:
            f *= (x-nodes[i]) / (nodes[k] - nodes[i])
    return f

def lagrange_interpolation(x, nodes, function_values):
    if len(nodes) != len(function_values):
        raise("Error: Nodes list must be as long as function_values")
    f = 0 # Neutral element for addition
    for i in range(0, len(nodes)):
        f += function_values[i] * lagrange(x, nodes, i)
    return f

For this assignment, we'll be numerically measuring the approximation error by using a discrete version of the Chebyshev distance (maximum metric). The interval given is $[a,b] = [-5,5]$

$$
||f||_h := \max\limits_{i \in \{0,...,1000\}} \Bigg | f \Bigg (a+\frac{i(b-a)}{1000} \Bigg ) \Bigg |
$$

Furthermore, we'll use the equidistant nodes

$$
x_i = a + \frac{i(b-a)}{n}, \text{ for } i \in {0,...,n}
$$

### a)
### i)
$$
f(x) := sin(x)
$$

We'll analyze the approximation error $||f-p_n||_h$ for $n \rightarrow \infty$ using $n \in \{1,...,100\}$

First, let's create our discrete Chebyshev distance function

In [167]:
# Interval
a,b = -5,5

def sin(x):
    return np.sin(x)

def generate_interval(n):
    interval = []
    for i in range(n):
        point = a + (i * (b - a) / n)
        interval.append(point)
    return np.array(interval)

# Returns ||f - p_n||_h
def chebyshev(a, b, func, x_nodes):
    
    y_nodes = func(x_nodes)

    interval = generate_interval(1000)

    interval_y = func(interval)
    return np.max(np.abs(interval_y - lagrange_interpolation(interval, x_nodes, y_nodes)))

Then, our n will run from 1 to 100 and approximate the error in each cycle according to our custom Chebyshev distance function.

In [168]:
def run_ns(func):
    for n in range(1,101):
        x_nodes = [None] * (n)
        x_nodes = generate_interval(n)
        max = chebyshev(a, b, sin, x_nodes)

run_ns(sin)



### b)

Now we'll do the exact same thing, but with the function

$$
f(x)=(1+x^2)^{-1}
$$